In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn. metrics import confusion_matrix, ConfusionMatrixDisplay , precision_score, recall_score, f1_score, r2_score, roc_curve, roc_auc_score, classification_report

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# feature_label_matrix = pd.read_csv('./labeled_feature_matrix.csv')
# display(feature_label_matrix.shape)

In [ ]:
# feature_matrix = np.loadtxt('./labeled_feature_matrix.csv', delimiter=',')
# display(feature_matrix.shape)

In [37]:
# Run this cell when executing Preprocessing.v1
feature_matrix = np.load("../outputs-v1/labeled_feature_vector.npy")

In [46]:
# Run this cell when executing preprocessing.v2 with filtering
feature_matrix = np.load("../codes-v2/labeled_feature_v2.npy")

In [125]:
# Run this cell when doing feature extraction specific to features
feature_matrix = np.load("../codes-v2/labeled_feature_rest-2-all.npy")

In [2]:
# Run this cell when doing feature extraction specific to features for all subjects
feature_matrix = np.load("../all-sub-features/rest-2-wavelet.npy")

In [32]:
# Run this cell when executing preprocessing.v2 with four class
feature_matrix = np.load("../codes-v2/labeled_feature_v3.npy")

In [34]:
# Run this cell when checking metrics for test-subjects
subject_number = 'onefive'

test_features = np.load(f"../test-subjects/wavelet-sub-{subject_number}.npy")

In [35]:
xTest = test_features[:, :-1]
display(xTest.shape)
for i in range(xTest.shape[0]):
        for j in range(xTest.shape[1]):
            if np.isnan(xTest[i,j]):
                  display([i,j])

(2834, 98)

In [36]:
yTest = test_features[:, -1]
display(yTest.shape)

(2834,)

In [37]:
display(feature_matrix.shape)

(24354, 99)

In [38]:
labels = feature_matrix[:, -1]
display(labels.shape)

(24354,)

In [39]:
print((np.unique(labels)))

[0. 1.]


In [40]:
data = feature_matrix[:, :-1]
display(data.shape)
for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if np.isnan(data[i,j]):
                  display([i,j])

(24354, 98)

In [41]:
fold = StratifiedKFold(n_splits=10)

In [42]:
for train_index, test_index in fold.split(data, labels):
    x_train, x_test, y_train, y_test = [data[i] for i in train_index], [data[i] for i in test_index], [labels[i] for i in train_index], [labels[i] for i in test_index]


# x_train, x_test, y_train, y_test = train_test_split(data,labels,test_size=0.3)

In [43]:
print(len(np.unique(y_test)))

2


### Plot Confusion Matrix Function

In [14]:
def plot_cnf(cnf_matrix):
  fig = px.imshow(cnf_matrix, 
                 color_continuous_scale='Blues')

  fig.update_layout(
      title="Confusion Matrix with Rest-case-labeled: 0, One-back-labeled: 1",
      xaxis_title="Actual Labels",
      yaxis_title="Predicted Labels",
      width= 700,
      height=700,
  )

  fig.update_layout(
      font=dict(
          size=12
      ),
      xaxis = dict(
          tick0=0,
          dtick=1
      ),
      yaxis = dict(
          tick0=0,
          dtick=1
      )
  )

  for i in range(len(cnf_matrix)):
      for j in range(len(cnf_matrix)):
          
          if (cnf_matrix[i,j] >= 220):
              color = 'white'
          else: 
              color = 'black'

          fig.add_annotation(text=str(cnf_matrix[i,j]), 
                            x=j, 
                            y=i, 
                            showarrow=False,
                            font=dict(
                                color=color,
                                size=24,
                            ) 
                            )

  fig.show()

### Plot ROC Curve Function

In [76]:
def plot_roc(tpr, fpr):
  
  fig = px.line(x=fpr, 
                y=tpr,
                )
    
  fig.add_scatter(x=[0,1], 
                  y=[0,1],
                  line=dict(color='navy', dash='dash'),
                  name="Guessing"
                  )

  
  fig.update_layout(
      title='ROC Curve',
      xaxis_title='False Positive Rate',
      yaxis_title='True Positive Rate',
      width=700,
      height=500,
  )

  fig.show()

### Plot multiple ROC plots

In [79]:
# Function to plot multiple ROC curevs on single plot

def multi_roc_plot(models, x_train= x_train, y_train= y_train, x_test=x_test, y_test= y_test):
  fpr_list = [] 
  tpr_list = []
  legend_list = []
  
  for model in models:
    # Training the models on x_train and y_train
    model.fit(x_train, y_train)
    
    y_pred_prob = model.predict_proba(X=x_test)         # y_pred_prob.shape = [len(x-test), 2], (prob(label0), prob(label1))

    # Getting the FalsePositiveRate and TruePositveRates from plotting the ROC curve
    fpr, tpr,_ = roc_curve(
                          y_test, 
                          y_pred_prob[:,1],
                          )
    
    tpr_list.append(tpr)
    fpr_list.append(fpr)

    index = str(model).find('(')
    legend_list.append(str(model)[:index])

  colors = px.colors.qualitative.Set1

  fig = px.line()

  for i, (tpr, fpr) in enumerate(zip(tpr_list, fpr_list)):
    fig.add_scatter(
                    x=fpr,
                    y=tpr,
                    mode='lines',
                    line=dict(color= colors[i]),
                    name= legend_list[i]
                    )
    
  fig.add_scatter(x=[0,1], 
                  y=[0,1],
                  line=dict(color='navy', dash='dash'),
                  name="Guessing"
                  )
  
  fig.update_layout(
      title='ROC Curve',
      xaxis_title='False Positive Rate',
      yaxis_title='True Positive Rate',
      width=700,
      height=500,
  )

  fig.show()

### Metrics Function

In [27]:
# Performance metrics calculation function

def metrics(model, labels, xTest=None, yTest=None):

  if np.all(xTest == None):
    # Using average of cross val score for accuracy
    score = cross_val_score(model, 
                            data, 
                            labels,
                            cv=10,
                          )
    
    cross_val_acc = np.average(score)

  # Training the model on x_train and y_train
  model.fit(x_train, y_train)

  if np.all(xTest == None):
    # Getting the class-label predictions and class-label prediction probabilities from the trained model
    model_predictions = model.predict(X=x_test)         # model_predictions.shape = [len(x-test)], (label(ith epoch))
    y_pred_prob = model.predict_proba(X=x_test)         # y_pred_prob.shape = [len(x-test), 2], (prob(label0), prob(label1))
  else:
    # Getting the class-label predictions and class-label prediction probabilities for the test data
    model_predictions = model.predict(X=xTest)         # model_predictions.shape = [len(x-test)], (label(ith epoch))
    y_pred_prob = model.predict_proba(X=xTest)         # y_pred_prob.shape = [len(x-test), 2], (prob(label0), prob(label1))

  if np.all(xTest == None):
    # Building the Classification Report using the predictions as a dataframe without the accuracy column
    classif_report = pd.DataFrame(classification_report(y_true=y_test, 
                                                        y_pred=model_predictions, 
                                                        output_dict=True,
                                                        zero_division=0,
                                                        )
                                  ).drop(labels="accuracy", axis=1).T.round(2)
  else:
    # Building the Classification Report using the predictions as a dataframe with the accuracy column
    classif_report = pd.DataFrame(classification_report(y_true=yTest, 
                                                        y_pred=model_predictions, 
                                                        output_dict=True,
                                                        zero_division=0,
                                                        )
                                  ).T.round(2)
  if np.all(xTest == None):  
    # Building the Confusion Matrix using the predicted class labels
    cnf_matrix = confusion_matrix(y_true=y_test, y_pred=model_predictions)
  else:
    # Building the Confusion Matrix using the predicted class labels for test-subjects
    cnf_matrix = confusion_matrix(y_true=yTest, y_pred=model_predictions)
  
  if len(np.unique(labels)) == 2:
    if np.all(xTest == None):
      # Getting the FalsePositiveRate and TruePositveRates from plotting the ROC curve
      fpr, tpr, thresholds = roc_curve(
                                      y_test, 
                                      y_pred_prob[:,1],
                                      #  pos_label=3
                                      )
    else:
      # Getting the FalsePositiveRate and TruePositveRates from plotting the ROC curve for test-subjects
      fpr, tpr, thresholds = roc_curve(
                                      yTest, 
                                      y_pred_prob[:,1],
                                      #  pos_label=3
                                      )

  if len(np.unique(labels)) == 2:
    if np.all(xTest == None):
      # Calculating the Area under the ROC curve ie, AUC using class label prediction probabilities
      auc = roc_auc_score(y_test, y_pred_prob[:,1])
    else:
      # Calculating the Area under the ROC curve ie, AUC using class label prediction probabilities for test-subjects
      auc = roc_auc_score(yTest, y_pred_prob[:,1])

  else:
    micro_roc_auc_ovr = roc_auc_score(
                                      y_test,
                                      y_pred_prob,
                                      multi_class="ovr",
                                      average="micro",
                                      )
    
    macro_roc_auc_ovr = roc_auc_score(
                                      y_test,
                                      y_pred_prob,
                                      multi_class="ovr",
                                      average="macro",
                                      )
    
    macro_roc_auc_ovo = roc_auc_score(
                                      y_test,
                                      y_pred_prob,
                                      multi_class="ovo",
                                      average="macro",
                                      )
  if np.all(xTest == None):  
    print(f"Cross-val-mean-Accuracy: {100*cross_val_acc:.2f}\n")

  display(classif_report)

  plot_cnf(cnf_matrix=cnf_matrix)

  if len(np.unique(labels)) == 2:
    plot_roc(tpr=tpr,
            fpr=fpr,
            )
  
  if len(np.unique(labels)) == 2:
    print(f"Area Under the ROC Curve (AUC): {100*auc:.2f}")
  
  else:
    print(f"\nMicro-averaged One-vs-Rest ROC AUC score:{micro_roc_auc_ovr:.2f}")
    print(f"\nMacro-averaged One-vs-Rest ROC AUC score:{macro_roc_auc_ovr:.2f}")
    print(f"\nMacro-averaged One-vs-One ROC AUC score:{macro_roc_auc_ovo:.2f}")

### Gaussian naive bayes

In [41]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [42]:
metrics(gnb, labels)

Cross-val-mean-Accuracy: 0.6226



,precision,recall,f1-score,support
0.0,0.49,0.97,0.65,417.0
2.0,0.35,0.02,0.03,422.0
macro avg,0.42,0.49,0.34,839.0
weighted avg,0.42,0.49,0.34,839.0


Area Under the ROC Curve (AUC): 0.4391


### Linear Regression

In [18]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', multi_class='auto')

In [19]:
metrics(lr, labels)

Cross-val-mean-Accuracy: 0.5026



,precision,recall,f1-score,support
0.0,0.00,0.0,0.00,417.0
3.0,0.50,1.0,0.67,422.0
macro avg,0.25,0.5,0.33,839.0
weighted avg,0.25,0.5,0.34,839.0


Area Under the ROC Curve (AUC): 0.5000


### XGB classifier

In [46]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [47]:
metrics(
        xgb, 
        labels, 
        # xTest=xTest, 
        # yTest=yTest        
        )

Cross-val-mean-Accuracy: 91.27



,precision,recall,f1-score,support
0.0,0.92,0.85,0.88,1209.0
1.0,0.86,0.92,0.89,1226.0
macro avg,0.89,0.89,0.89,2435.0
weighted avg,0.89,0.89,0.89,2435.0


Area Under the ROC Curve (AUC): 94.26


### XGB Model metric automator

In [11]:
feature_list = ["rest-2-all.npy","rest-2-psd-hjorth.npy","rest-2-psd-kurtosis.npy","rest-2-psd.npy","rest-2-wavelet-hjorth-kurtosis.npy","rest-2-wavelet-hjorth.npy","rest-2-wavelet-kurtosis.npy","rest-2-wavelet-psd-hjorth.npy","rest-2-wavelet-psd-kurtosis.npy","rest-2-wavelet-psd.npy","rest-2-wavelet.npy"]

In [12]:
for feature_file in feature_list:

    feature_matrix = np.load(f"../all-sub-features/{feature_file}")

    labels = feature_matrix[:, -1]
    data = feature_matrix[:, :-1]

    print(f"Shape of {feature_file}:")
    print(data.shape)


    for train_index, test_index in fold.split(data, labels):

        x_train, x_test, y_train, y_test = data[train_index], data[test_index], labels[train_index], labels[test_index]

    from xgboost import XGBClassifier
    xgb = XGBClassifier()

    metrics(xgb, labels)


Shape of rest-2-all.npy:
(24354, 130)
Cross-val-mean-Accuracy: 90.04



,precision,recall,f1-score,support
0.0,0.87,0.85,0.86,1209.0
1.0,0.86,0.87,0.86,1226.0
macro avg,0.86,0.86,0.86,2435.0
weighted avg,0.86,0.86,0.86,2435.0


Area Under the ROC Curve (AUC): 92.40
Shape of rest-2-psd-hjorth.npy:
(24354, 31)
Cross-val-mean-Accuracy: 61.71



,precision,recall,f1-score,support
0.0,0.60,0.54,0.56,1209.0
1.0,0.58,0.64,0.61,1226.0
macro avg,0.59,0.59,0.59,2435.0
weighted avg,0.59,0.59,0.59,2435.0


Area Under the ROC Curve (AUC): 61.11
Shape of rest-2-psd-kurtosis.npy:
(24354, 29)
Cross-val-mean-Accuracy: 59.38



...

Area Under the ROC Curve (AUC): 91.35
Shape of rest-2-wavelet-psd.npy:
(24354, 126)
Cross-val-mean-Accuracy: 90.12



,precision,recall,f1-score,support
0.0,0.86,0.85,0.85,1209.0
1.0,0.86,0.86,0.86,1226.0
macro avg,0.86,0.86,0.86,2435.0
weighted avg,0.86,0.86,0.86,2435.0


Area Under the ROC Curve (AUC): 91.58
Shape of rest-2-wavelet.npy:
(24354, 98)
Cross-val-mean-Accuracy: 91.27



,precision,recall,f1-score,support
0.0,0.92,0.85,0.88,1209.0
1.0,0.86,0.92,0.89,1226.0
macro avg,0.89,0.89,0.89,2435.0
weighted avg,0.89,0.89,0.89,2435.0


Area Under the ROC Curve (AUC): 94.26


### ETC 

In [50]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=50)

In [73]:
metrics(etc, labels)

Cross-val-mean-Accuracy: 87.65



,precision,recall,f1-score,support
0.0,0.87,0.74,0.80,1209.0
1.0,0.77,0.89,0.83,1226.0
macro avg,0.82,0.82,0.81,2435.0
weighted avg,0.82,0.82,0.81,2435.0


Area Under the ROC Curve (AUC): 87.10


### ETC model metrics automator

In [13]:
for feature_file in feature_list:

    feature_matrix = np.load(f"../all-sub-features/{feature_file}")

    labels = feature_matrix[:, -1]
    data = feature_matrix[:, :-1]

    print(f"Shape of {feature_file}:")
    print(data.shape)


    for train_index, test_index in fold.split(data, labels):

        x_train, x_test, y_train, y_test = data[train_index], data[test_index], labels[train_index], labels[test_index]

    from sklearn.ensemble import ExtraTreesClassifier
    etc = ExtraTreesClassifier(n_estimators=50)

    metrics(etc, labels)

Shape of rest-2-all.npy:
(24354, 130)
Cross-val-mean-Accuracy: 87.54



,precision,recall,f1-score,support
0.0,0.88,0.75,0.81,1209.0
1.0,0.78,0.90,0.84,1226.0
macro avg,0.83,0.82,0.82,2435.0
weighted avg,0.83,0.83,0.82,2435.0


Area Under the ROC Curve (AUC): 88.10
Shape of rest-2-psd-hjorth.npy:
(24354, 31)
Cross-val-mean-Accuracy: 56.81



,precision,recall,f1-score,support
0.0,0.57,0.56,0.57,1209.0
1.0,0.58,0.59,0.58,1226.0
macro avg,0.58,0.58,0.58,2435.0
weighted avg,0.58,0.58,0.58,2435.0


Area Under the ROC Curve (AUC): 59.07
Shape of rest-2-psd-kurtosis.npy:
(24354, 29)
Cross-val-mean-Accuracy: 50.95



...

Area Under the ROC Curve (AUC): 88.65
Shape of rest-2-wavelet-psd.npy:
(24354, 126)
Cross-val-mean-Accuracy: 88.27



,precision,recall,f1-score,support
0.0,0.88,0.75,0.81,1209.0
1.0,0.78,0.90,0.84,1226.0
macro avg,0.83,0.82,0.82,2435.0
weighted avg,0.83,0.82,0.82,2435.0


Area Under the ROC Curve (AUC): 88.00
Shape of rest-2-wavelet.npy:
(24354, 98)
Cross-val-mean-Accuracy: 88.35



,precision,recall,f1-score,support
0.0,0.89,0.76,0.82,1209.0
1.0,0.79,0.91,0.85,1226.0
macro avg,0.84,0.83,0.83,2435.0
weighted avg,0.84,0.83,0.83,2435.0


Area Under the ROC Curve (AUC): 89.03


### RFC

In [51]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50)

In [74]:
metrics(rfc, labels)

Cross-val-mean-Accuracy: 88.26



,precision,recall,f1-score,support
0.0,0.87,0.7,0.78,1209.0
1.0,0.75,0.9,0.82,1226.0
macro avg,0.81,0.8,0.80,2435.0
weighted avg,0.81,0.8,0.80,2435.0


Area Under the ROC Curve (AUC): 87.59


### RFC model metrics automator


In [14]:
for feature_file in feature_list:

    feature_matrix = np.load(f"../all-sub-features/{feature_file}")

    labels = feature_matrix[:, -1]
    data = feature_matrix[:, :-1]

    print(f"Shape of {feature_file}:")
    print(data.shape)


    for train_index, test_index in fold.split(data, labels):

        x_train, x_test, y_train, y_test = data[train_index], data[test_index], labels[train_index], labels[test_index]

    from sklearn.ensemble import RandomForestClassifier
    rfc = RandomForestClassifier(n_estimators=50)

    metrics(rfc, labels)

Shape of rest-2-all.npy:
(24354, 130)
Cross-val-mean-Accuracy: 87.88



,precision,recall,f1-score,support
0.0,0.88,0.72,0.79,1209.0
1.0,0.76,0.90,0.83,1226.0
macro avg,0.82,0.81,0.81,2435.0
weighted avg,0.82,0.81,0.81,2435.0


Area Under the ROC Curve (AUC): 88.13
Shape of rest-2-psd-hjorth.npy:
(24354, 31)
Cross-val-mean-Accuracy: 56.94



,precision,recall,f1-score,support
0.0,0.59,0.58,0.59,1209.0
1.0,0.59,0.60,0.59,1226.0
macro avg,0.59,0.59,0.59,2435.0
weighted avg,0.59,0.59,0.59,2435.0


Area Under the ROC Curve (AUC): 59.96
Shape of rest-2-psd-kurtosis.npy:
(24354, 29)
Cross-val-mean-Accuracy: 50.87



...

Area Under the ROC Curve (AUC): 88.44
Shape of rest-2-wavelet-psd.npy:
(24354, 126)
Cross-val-mean-Accuracy: 81.26



,precision,recall,f1-score,support
0.0,0.76,0.75,0.75,1209.0
1.0,0.75,0.76,0.76,1226.0
macro avg,0.76,0.76,0.76,2435.0
weighted avg,0.76,0.76,0.76,2435.0


Area Under the ROC Curve (AUC): 83.25
Shape of rest-2-wavelet.npy:
(24354, 98)
Cross-val-mean-Accuracy: 83.87



,precision,recall,f1-score,support
0.0,0.82,0.73,0.77,1209.0
1.0,0.76,0.84,0.80,1226.0
macro avg,0.79,0.78,0.78,2435.0
weighted avg,0.79,0.78,0.78,2435.0


Area Under the ROC Curve (AUC): 85.39


In [73]:
models = [xgb, etc, rfc]
str_list = []
for model in models:
  index = str(model).find('(')
  str_list.append(str(model)[:index])

display(",".join(str_list))
# display(str([0][:]))

'XGBClassifier,ExtraTreesClassifier,RandomForestClassifier'

### XGB,ETC,RFC multi-roc plot

In [81]:
multi_roc_plot([rfc, etc, xgb])

### SVC

In [22]:
from sklearn.svm import SVC
svc = SVC(gamma='auto',
          probability=True,
          )

In [23]:
metrics(svc, labels)

Cross-val-mean-Accuracy: 0.5026



,precision,recall,f1-score,support
0.0,0.00,0.0,0.00,417.0
3.0,0.50,1.0,0.67,422.0
macro avg,0.25,0.5,0.33,839.0
weighted avg,0.25,0.5,0.34,839.0


Area Under the ROC Curve (AUC): 0.5000


### KNN 

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=27)

In [27]:
metrics(knn, labels)

Cross-val-mean-Accuracy: 0.2589



,precision,recall,f1-score,support
0.0,0.40,0.37,0.38,417.0
1.0,0.26,0.26,0.26,422.0
2.0,0.24,0.32,0.27,422.0
3.0,0.28,0.22,0.25,421.0
macro avg,0.30,0.29,0.29,1682.0
weighted avg,0.30,0.29,0.29,1682.0



Micro-averaged One-vs-Rest ROC AUC score:0.57

Macro-averaged One-vs-Rest ROC AUC score:0.57

Macro-averaged One-vs-One ROC AUC score:0.57
